**TensorFlow** 是目前業界常見的選擇之一，特別是與 Keras 結合使用時，它提供了強大的易用性和靈活性，因此我們將使用 TensorFlow 來實作這些指標，並評估 CNN 的效能。

這次的實作將展示如何計算精確度、召回率、F1 分數和 ROC 曲線，並對已經訓練好的 CNN 模型進行評估。


### **實作重點：**

1. **精確度、召回率、F1 分數：**
   使用 `classification_report` 可以快速得到這些指標，它會針對每個類別提供精確度、召回率、F1 分數的詳細報告。

2. **混淆矩陣：**
   使用 `confusion_matrix` 來了解模型的分類錯誤情況，混淆矩陣能直觀地顯示每個類別的預測結果與實際標籤的對應關係。

3. **ROC 曲線和 AUC：**
   為每個類別計算 ROC 曲線和 AUC，並用圖形展示其中一個類別的 ROC 曲線，幫助你視覺化模型的分類能力。

---

### **指標解釋：**

- **精確度（Precision）：** 模型預測為正類的樣本中實際正類的比例。適合在錯誤分類正類代價較高的情況下使用。
- **召回率（Recall）：** 在實際正類樣本中被正確預測為正類的比例。適合在錯誤分類負類代價較高的情況下使用。
- **F1 分數：** 精確度和召回率的調和平均，用來平衡兩者之間的衝突。
- **ROC 曲線與 AUC：** 用於評估模型在不同閾值下的分類性能，AUC 越接近 1，模型效果越好。

---

### **結論：**

通過這些評估指標，我們可以深入了解 CNN 模型的效能。根據不同任務的需求，你可以重點關注某些指標來優化模型。這些方法是業界常用的評估手段，有助於在實際應用中更好地選擇和調整模型。

### **TensorFlow 實作：評估 CNN 模型效能**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# 下載並預處理 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 將訓練和測試圖像數據重塑為 (28, 28, 1) 並正規化到 0-1 範圍
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255.0  # 增加通道維度並正規化
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255.0

# 建立 CNN 模型
model = models.Sequential([
    # 第一個卷積層，使用 32 個 3x3 的卷積核，激活函數為 ReLU，輸入形狀為 (28, 28, 1)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    # 最大池化層，池化窗口大小為 2x2
    layers.MaxPooling2D((2, 2)),
    # 第二個卷積層，使用 64 個 3x3 的卷積核，激活函數為 ReLU
    layers.Conv2D(64, (3, 3), activation='relu'),
    # 最大池化層，池化窗口大小為 2x2
    layers.MaxPooling2D((2, 2)),
    # 展開層，將多維數據展開為一維
    layers.Flatten(),
    # 全連接層，包含 128 個神經元，激活函數為 ReLU
    layers.Dense(128, activation='relu'),
    # 輸出層，包含 10 個神經元，對應 10 個類別，激活函數為 softmax
    layers.Dense(10, activation='softmax')
])

# 編譯模型
# 使用 Adam 優化器，損失函數為 sparse_categorical_crossentropy，評估指標為準確率
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
# 使用訓練數據進行訓練，訓練 5 個 epoch，每批次大小為 64
model.fit(x_train, y_train, epochs=5, batch_size=64)

# 預測測試數據
y_pred = model.predict(x_test)  # 預測測試數據
y_pred_classes = np.argmax(y_pred, axis=1)  # 取得預測類別，使用 argmax 獲取每個樣本的最大概率對應的類別
y_true = y_test  # 實際類別

In [ ]:
# 計算精確度、召回率、F1 分數
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

In [ ]:
# 計算混淆矩陣
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

In [ ]:
# 計算 ROC 曲線和 AUC
fpr = dict()  # 儲存每個類別的假陽性率
tpr = dict()  # 儲存每個類別的真陽性率
roc_auc = dict()  # 儲存每個類別的 AUC 值
n_classes = 10  # 類別數量，MNIST 資料集有 10 個類別

# 將類別轉換為 one-hot 編碼格式
# y_true 是實際的類別標籤，將其轉換為 one-hot 編碼格式
y_test_one_hot = tf.keras.utils.to_categorical(y_true, num_classes=n_classes)

# 計算每個類別的 ROC 曲線和 AUC 值
for i in range(n_classes):
    # roc_curve 函數計算假陽性率和真陽性率
    fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], y_pred[:, i])
    # auc 函數計算 AUC 值
    roc_auc[i] = auc(fpr[i], tpr[i])

# 繪製 ROC 曲線（僅繪製其中一個類別的示例）
plt.figure()
# 繪製類別 0 的 ROC 曲線
plt.plot(fpr[0], tpr[0], color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc[0]:.2f})')
# 繪製對角線，表示隨機猜測的 ROC 曲線
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])  # 設定 x 軸範圍
plt.ylim([0.0, 1.05])  # 設定 y 軸範圍
plt.xlabel('False Positive Rate')  # x 軸標籤
plt.ylabel('True Positive Rate')  # y 軸標籤
plt.title('Receiver Operating Characteristic - Class 0')  # 圖表標題
plt.legend(loc="lower right")  # 圖例位置
plt.show()  # 顯示圖表